<a href="https://colab.research.google.com/github/sjut/HSE-Compling/blob/master/seminars/6_NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural machine translation with attention

In [0]:
# можно примонтировать гуглдиск, чтобы хранить там промежуточные данные
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

## Подготовка данных

Будем использовать датасет [отсюда](http://www.manythings.org/anki/). Он содержит предложения и их переводы, выполненные пользователями проекта [Tatoeba](https://tatoeba.org/rus/sentences/search?from=und&to=und&query=may+i+borrow+this+book). Там собираются общеупотребительные фразы и варианты их перевода — своего рода краудсорс-разговорник.

```
May I borrow this book?	Я могу одолжить эту книгу?
```

Подготовка данных включает следующие шаги:

1. Добавление токенов начала и конца предложения — *start* и *end*.
2. Удаляем из предложений лишние символы.
3. Составляем индекс слов (словарь слово - id) и обратный индекс (id - слово).
4. Делаем padding — дописываем специальный токен в каждое предложение до максимальной длины.

In [0]:
# Скачиваем и распаковываем
!wget http://www.manythings.org/anki/rus-eng.zip
!unzip rus-eng.zip

In [0]:
path_to_file = "/content/rus.txt"

In [0]:
# Нормализация юникода
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = w.lower().strip()

  # Примитивная токенизация: отрезаем пунктуацию
  w = re.sub(r"([?.!,\:;\-—])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # Выкидываем всё кроме букв и основной пунктуации
  w = re.sub(r"[^a-zA-Zа-яА-Я?.!,\:;\-—]+", " ", w)

  w = w.rstrip().strip()

  # Добавляем токены начала и конца предложения
  # NB! стандартные обозначения (часто даже в статьях бывают без расшифровки) — BOS / EOS
  w = '<start> ' + w + ' <end>'
  return w

In [0]:
en_sentence = "May I borrow this book?"
sp_sentence = "Можно мне взять эту книгу?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence))

In [0]:
!head rus.txt

In [0]:
# Препроцессим и делаем списки предложений для каждого языка
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [0]:
en, ru = create_dataset(path_to_file, None)
print(en[-1])
print(ru[-1])

In [0]:
def max_length(tensor):
  return max(len(t) for t in tensor)

In [0]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [0]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

### Обучаемся на кусочке корпуса

За обозримое время попробуем что-то обучить на 30К предложений:

In [0]:
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

#Считаем максимальную длину, до которой нужно дополнить все предложения
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [0]:
# Делим на train и validation
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Проверим длины
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

In [0]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [0]:
# проверка, что индексация работает как надо
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

### Создаем tf-ный датасет

In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [0]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

## Описываем модель

Реализуем модель encoder-decoder c механизмом внимания (attention) примерно так же, как в [Neural Machine Translation (seq2seq) tutorial](https://github.com/tensorflow/nmt) от команды tensorflow. Внимание работает примерно так: каждому слову (subword unit'у) приписывается вес, который затем использует декодер, когда предсказывает следующее слово в предложении. Иллюстрация из [статьи Luong et al.](https://arxiv.org/abs/1508.04025v5) 

<img src="https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg" width="500" alt="attention mechanism">

Энкодер обрабатывает входные предложение, на выходе мы получаем матрицу *(batch_size, max_length, hidden_size)* и скрытое состояние энкодера размером *(batch_size, hidden_size)*.

Как получаются веса внимания:

<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_0.jpg" alt="attention equation 0" width="800">
<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_1.jpg" alt="attention equation 1" width="800">

Такой механизм описан в [статье Bahdanau et al.](https://arxiv.org/pdf/1409.0473.pdf). 

Как затем работает декодер?
Обозначения:

* FC = полносвязный слой
* EO = выход энкодера
* H = скрытое состояние энкодера
* X = вход декодера (слово на языке перевода)

Псевдокод:

* `score = FC(tanh(FC(EO) + FC(H)))`
* `attention weights = softmax(score, axis = 1)`. Размер score'а — *(batch_size, max_length, hidden_size)*. `Max_length` длина входного предложения. Поскольку хотим получить веса для всех входных слов, применяем `softmax` по первому измерению.
* `context vector = sum(attention weights * EO, axis = 1)`.
* `embedding output` = Вход декодера преобразуем в эмбеддинг.
* `merged vector = concat(embedding output, context vector)`
* Этот вектор подаем на вход GRU:
<img src="http://dprogrammer.org/wp-content/uploads/2019/04/RNN-vs-LSTM-vs-GRU-1200x361.png" alt="rnn lstm gru" width="800">

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [0]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [0]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

## Оптимизатор, функция потерь

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints

Полезно записывать, что происходит во время обучения (как меняются веса модели, loss и тп). Для этого заводим checkpoint'ы и указываем, куда писать всю эту информацию.

In [0]:
checkpoint_dir = './drive/My Drive/training_checkpoints_all'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## Обучение

1. Передаем *inp* в *encoder*, который возвращает *enc_output* и *enc_hidden*.
2. Эти вектора и вход декодера *dec_input* (*start token*) передаем в *decoder*.
3. Получаем *predictions* и *dec_hidden*.
4. Скрытое состояние декодера передаем дальше, а по *predictions* считаем значение функции потерь *loss*.
5. Чтобы взять следующий вход для декодера, используем технику *teacher forcing*: передаем *target word* как следующий *dec_input*.
6. Считаем градиенты, обновляем веса в нужную сторону.

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  # штука для автоматического дифференцирования
  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [0]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

## Transformer
На самом деле, модель, которую мы реализовали, довольно проста. Современные системы МП используют модель, которая называется Трансформер:

<img src="https://s72sas.storage.yandex.net/rdisk/339fd6c6f0ac653d592377c701a50c1fb255ea4d7b02074170d793b2f8356168/5de9600a/g4KF9Q7MSog3Y6TSUPRQ4Ame_ZHqZ_8w-InZKcFhhdn28KCuHRTqix_qT7NwWlLHFSBrJ1sDjeaTyQUin1VWQA==?uid=159590558&filename=transformer.png&disposition=inline&hash=&limit=0&content_type=image%2Fpng&owner_uid=159590558&fsize=227923&hid=5e8eeb4717ce2cae156ee4031cbbfa8d&media_type=image&tknv=v2&etag=9d9d2d12dc620a5e3ccc248d32880970&rtoken=TldHZ2QcKCOO&force_default=yes&ycrid=na-c619aaa0ae7799716a9d81e851c9bbe1-downloader24h&ts=598fa45b09680&s=987263a9e385189bf82cfb56cddadfff8ebcfa43335ec0637ad7888b72ab3971&pb=U2FsdGVkX19q59k9bM_wKSOmq3NpJ4HHQdC0GZyTII92KHYep4hTr-oepmPS7XnesjRl7ESfIPjhtkZ600uIvk21lYff0Fu7brh6VKBFwCs" alt="transformer" width="500">

Она описана в [статье Attention is all you need!](https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf)

Существует множество исследований о том, что происходит внутри трансформера, я рекомендую вот [эти](https://lena-voita.github.io/posts.html) (в постах есть ссылки на сами статьи).

## Перевод

* Заводим функцию *evaluate*, которая повторяет то же, что мы делаем во время обучения, но уже без *teacher forcing*. На вход декодеру всегда подаем его предыдущий выход, выход енкодера и его скрытое состояние.
* Перестаем генерировать перевод, когда получаем *end token*.
* Храним веса внимания *attention weights* на каждом шаге.

In [0]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [0]:
# Функция, которая будет рисовать attention, похожий на alignment
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [0]:
# переводим, выводим результат и attention plot
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  plot_attention(attention_plot, sentence.split(' '), result.split(' '))

## Тестируем

In [0]:
# Восстановим модель из checkpoint'а
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
translate('здесь очень холодно.')

In [0]:
translate('такова жизнь.')

In [0]:
# модель начинает сочинять
translate('ты сегодня дома?')

In [0]:
translate('чашку кофе, пожалуйста.')

## Тестируем модель на "большом" корпусе

Для этого нужно заново завести encoder, decoder, optimizer — с соответствующим словарем.

In [0]:
!wget "https://downloader.disk.yandex.ru/disk/7c8b616116f5eed6db9a0ce82ff4802cf49d80a4d3fec9898c6470cfc70cd773/5de9651d/g4KF9Q7MSog3Y6TSUPRQ4PyJU-AVojbOn-7qhBVqP-XLW8qDsN9UjtD4f2R85FdHdwZL9mNCBT99kgrSHO_luQ%3D%3D?uid=0&filename=training_checkpoints_all.zip&disposition=attachment&hash=P4A3aY7el7MX/aYW2SeFB9/y/mzUg57SmmpqtY28miCejIKYnz16blPfICxhEBbnq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=159590558&fsize=399529572&hid=83a651136c7b105699e6d34a9870eacc&media_type=compressed&tknv=v2" -O training_checkpoints_all.zip

In [0]:
!mkdir training_checkpoints_all
!unzip training_checkpoints_all.zip -d training_checkpoints_all

In [0]:
## YOUR CODE HERE
# num_examples = 30000
# input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)
# ...
# encoder = Encoder(...)

In [0]:
checkpoint.restore(tf.train.latest_checkpoint("training_checkpoints_all"))

In [0]:
translate('ты сегодня дома?')